In [3]:
import sys
import os
import glob
import librosa
import numpy as np
import librosa.display
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
# go to upper diretory
sys.path.append(os.path.abspath('./../../'))
from Audio_Sentiment_Analysis.utils.Configuration import Configuration

In [7]:
AUDIO_DIR = f"{os.path.abspath('./../../')}/IEMOCAP_Dataset"
EXTRACTED_FEATURES_FILE = 'extracted_features_iemocap.csv'
CONFIG_FILE = f"{os.path.abspath('./../../')}/Audio_Sentiment_Analysis/iemocap_data/config.json"
config = Configuration.load_json(CONFIG_FILE)

## Extracting the 24 features selected from the eNTERFACE05 dataset study

In [8]:
chosen_features = {'var_mfcc15', 'min_mfcc17', 'std_chroma_stft', 'mean_zcr',
    'min_zcr', 'var_mfcc4', 'spikes_spec_cent', 'var_spec_bw', 'max_mfcc6',
    'min_mfcc19', 'max_mfcc13', 'max_mfcc5', 'var_mfcc9', 'min_mfcc7', 'var_mfcc1',
    'max_mfcc1', 'max_mfcc9', 'var_mel_spect', 'mean_spec_cont', 'var_mfcc2', 'max_mfcc10',
    'min_spec_cent', 'var_mfcc14', 'var_mfcc3'}

In [9]:
headers = [
    'File', 'Interaction', 'Gender', 'Duration', 'Emotion', 'Emotion_Id',
    'Valence', 'Activation', 'Dominance',
    'std_chroma_stft', 'mean_zcr', 'min_zcr',
    'var_mel_spect', 'spikes_spec_cent', 'min_spec_cent',
    'var_spec_bw', 'mean_spec_cont', 
    'var_mfcc1', 'max_mfcc1', 'var_mfcc2', 'var_mfcc3', 'var_mfcc4',
    'max_mfcc5', 'max_mfcc6', 'min_mfcc7', 'var_mfcc9', 'max_mfcc9',
    'max_mfcc10', 'max_mfcc13', 'var_mfcc14', 'var_mfcc15', 
    'min_mfcc17', 'min_mfcc19'
]

In [10]:
def spikes(data):
    if len(data.shape) != 1:
        data = np.concatenate(data)
    mean = np.mean(data)
    std = np.std(data)
    threshold = mean + std * 2 / 100
    num_spikes = 0
    for value in data:
        if value >= threshold:
            num_spikes += 1

    return num_spikes

In [11]:
emotion_label = {
    'ang': "angry",
    'hap': "happy",
    'sad': "sad",
    'neu': "neutral",
    'fru': "frustrated",
    'exc': "excited",
    'fea': "fearful",
    'sur': "surprised",
    'dis': "disgusted",
    'xxx': "other",
    'oth': "other"
}

emotion_number = {
    'ang': 0,
    'hap': 1,
    'sad': 2,
    'neu': 3,
    'fru': 4,
    'exc': 5,
    'fea': 6,
    'sur': 7,
    'dis': 8,
    'oth': 9,
    'xxx': 10,
}

In [13]:
def extract_features(audio_file, duration, emotion, valence, activation, dominance, writer):
    # file = audio_file.split(".")[-2].split("\\")[-1]
    file = audio_file.split(".")[-2].split("/")[-1]
    acting = "improvisation" if file.split("_")[1][:6] == "improv" else "scripted"
    gender = "Male" if file.split("_")[-1][0] == 'M' else "Female"

    y, sr = librosa.load(audio_file, res_type='kaiser_fast')

    std_chroma_stft = np.std(librosa.feature.chroma_stft(y, sr=sr))
    zcr = librosa.feature.zero_crossing_rate(y)
    mean_zcr = np.mean(zcr)
    min_zcr = np.min(zcr)
    var_mel_spect = np.var(librosa.feature.melspectrogram(y, sr=sr, n_mels=config.n_mels))
    spec_cent = librosa.feature.spectral_centroid(y, sr=sr)
    spikes_spec_cent = spikes(spec_cent)
    min_spec_cent = np.min(spec_cent)
    var_spec_bw = np.var(librosa.feature.spectral_bandwidth(y, sr=sr))
    mean_spec_cont = np.mean(librosa.feature.spectral_contrast(y, sr=sr))
    mfcc = librosa.feature.mfcc(y, sr=sr)
    var_mfcc1 = np.var(mfcc[0])
    max_mfcc1 = np.max(mfcc[0])
    var_mfcc2 = np.var(mfcc[1])
    var_mfcc3 = np.var(mfcc[2])
    var_mfcc4 = np.var(mfcc[3])
    max_mfcc5 = np.max(mfcc[4])
    max_mfcc6 = np.max(mfcc[5])
    min_mfcc7 = np.min(mfcc[6])
    var_mfcc9 = np.var(mfcc[8])
    max_mfcc9 = np.max(mfcc[8])
    max_mfcc10 = np.max(mfcc[9])
    max_mfcc13 = np.max(mfcc[12])
    var_mfcc14 = np.var(mfcc[13])
    var_mfcc15 = np.var(mfcc[14])
    min_mfcc17 = np.min(mfcc[16])
    min_mfcc19 = np.min(mfcc[18])

    features_str = f'{file} {acting} {gender} {duration} {emotion_label[emotion]} {emotion_number[emotion]}\
        {valence} {activation} {dominance}\
        {std_chroma_stft} {mean_zcr} {min_zcr} {var_mel_spect} {spikes_spec_cent} {min_spec_cent}\
        {var_spec_bw} {mean_spec_cont} {var_mfcc1} {max_mfcc1} {var_mfcc2} {var_mfcc3} {var_mfcc4}\
        {max_mfcc5} {max_mfcc6} {min_mfcc7} {var_mfcc9} {max_mfcc9} {max_mfcc10} {max_mfcc13}\
        {var_mfcc14} {var_mfcc15} {min_mfcc17} {min_mfcc19}'

    writer.writerow(features_str.split())

In [38]:
def process_data(audio_dir, headers, proc_feat_dataset):
    # Create a CSV for storing all processed features and write the header
    file = open(proc_feat_dataset, 'w', newline='')
    writer = csv.writer(file)
    writer.writerow(headers)

    print("Processing audio files from all dialogs:")
    for file_path in tqdm(glob.glob(audio_dir+'/Session[0-9]*/dialog/EmoEvaluation/*.txt')):
        emoEvalFile = open(file_path)

        for line in emoEvalFile:
            if line[0] == '[':
                args = line.split()
                duration = float(args[2][:-1]) - float(args[0][1:])
                '/Users/mario/Desktop/VADER/IEMOCAP_Dataset/Session4/sentences/wav/t/Ses04F_script02_1_F000.wav'
                audio_file = f'{AUDIO_DIR}/Session{args[3][4]}/sentences/wav/{file_path.split("/")[-1][:-4]}/{args[3]}.wav'
                extract_features(audio_file, duration, args[4], args[5][1:-1], args[6][:-1], args[7][:-1], writer)

In [39]:
process_data(AUDIO_DIR, headers, EXTRACTED_FEATURES_FILE)

Processing audio files from all dialogs:


100%|██████████| 151/151 [29:56<00:00, 11.89s/it]
